# Building Abstractions with Procedures

## Exercise 1.1
Below is a sequence of expressions. What is the result printed by the interpreter in response to each expression? Assume that the sequence is to be evaluated in the order in which it is presented
```racket
10
(+ 5 3 4)
(- 9 1)
(/ 6 2)
(+ (* 2 4) (- 4 6))
(define a 3)
(define b (+ a 1))
(+ a b (* a b))
(= a b)
(if (and (> b a) (< b (* a b)))
    b
    a)
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))
(+ 2 (if (> b a) b a))
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))
```

In [27]:
10

10

In [28]:
(+ 5 3 4)

12

In [29]:
(- 9 1)

8

In [30]:
(/ 6 2)

3

In [31]:
(+ (* 2 4) (- 4 6))

6

In [32]:
(define a 3)
(define b (+ a 1))
(+ a b (* a b))

19

In [33]:
(= a b)

#f

In [34]:
(if (and (> b a) (< b (* a b)))
    b
    a)

4

In [35]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))

16

In [36]:
(+ 2 (if (> b a) b a))

6

In [37]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))

16

## Exercise 1.2
Translate the following expression into prefix form
$$
\frac{5+4+(2-(3-(6+\frac{4}{3})))}{3(6-2)(2-7)}
$$

In [38]:
(/ (+ 5 4 (- 2
             (- 3
                (+ 6 (/ 4 3)))))
   (* 3 (- 6 2) (- 2 7)))

-23/90

## Exercise 1.3
Define a procedure that takes three numbers as arguments and returns the sum of the squares of the two larger numbers.

In [39]:
(define (square x)
  (* x x))
(define (sum_of_squares a b)
  (+ (square a) (square b)))
(define (sum_square_of_two_larger x y z)
  (cond ((and (> x z) (> y z))
        (sum_of_squares x y))
        ((> x y)
         (sum_of_squares x z))
        (else (sum_of_squares y z))))

## Exercise 1.4
Observe that our model of evaluation allows for combinations whose operators are compound expressions. Use this observation to describe the behavior of the following procedure:
```racket
(define (a-plus-abs-b a b)
  ((if (> b 0) + -) a b))
```

When we evalute `(a-plus-abs-b a b)`, operator part `(if (> b 0) + -)` will be evaluted first. If `b` is positive, then `(+ a b)` will be evaluated. If `b` is negative, then `(- a b)` will be evaluated.

## Exercise 1.5
Ben Bitdiddle has invented a test to determine whether the interpreter he is faced with is using applicative-order evaluation or normal-order evaluation. He defines the following two procedures:
```racket
(define (p) (p))

(define (test x y)
  (if (= x 0)
      0
      y))
```
Then he evaluates the expression
```racket
(test 0 (p))
```
What behavior will Ben observe with an interpreter that uses applicative-order evaluation? What behavior will he observe with an interpreter that uses normal-order evaluation? Explain your answer. (Assume that the evaluation rule for the special form `if` is the same whether the interpreter is using normal or applicative order: The predicate expression is evaluated first, and the result determines whether to evaluate the consequent or the alternative expression.)

Applicative-order:
The operator with `if` will be evalute first, then we get 0 directly.
Normal-order:
To substitute `(p)`, we get another `(p)` in `(test 0 (p))` , it's a infinite loop. Without primitive process we cannot go forward. 